# Using the Python `chi` library

## Getting started

Before you do anything, you should first set up a few project variables. This is so your script knows which project to operate on. If you are only a member of one project, you can skip this step, as the project will be selected for you by default. You can also set the site you want to use via the `region_name` setting - this defaults to `CHI@UC`.

You can get the your list of active projects from https://chameleoncloud.org/user/dashboard/

The "project_name" is of the form `CHI-XXXXXX`

In [1]:
import chi
# chi.set('project_name', '<CHI-XXXXXX>')    # Replace with your project ID from https://chameleoncloud.org/user/dashboard/
chi.use_site('CHI@Purdue')    #Set which site to use

Now using CHI@Purdue:
URL: https://chi.rcac.purdue.edu
Location: None
Support contact: help@chameleoncloud.org


## Launching a baremetal server

Starting a new baremetal server is easy to do.

In [ ]:
import chi.lease
# Reserve a head node
head_node_reservation_list = []
chi.lease.add_node_reservation(head_node_reservation_list, node_type='indyscc_head',count=1)
head_lease = chi.lease.create_lease("my-indyscc-head-node-lease", reservations=head_node_reservation_list)

print("see https://chi.rcac.purdue.edu/project/leases/ for current lease status")
# wait for lease to become active
head_lease_start = chi.lease.wait_for_active(head_lease['id'])  # Ensure lease has started

In [ ]:
# Reserve 10 compute nodes
compute_node_reservation_list = []
chi.lease.add_node_reservation(compute_node_reservation_list, node_type='indyscc_compute',count=10)
compute_lease = chi.lease.create_lease("my-indyscc-compute-node-lease", reservations=compute_node_reservation_list)

print("see https://chi.rcac.purdue.edu/project/leases/ for current lease status")
# wait for lease to become active
compute_lease_start = chi.lease.wait_for_active(compute_lease['id'])  # Ensure lease has started

In [ ]:
import chi.server

# Launch an instance on your head node
head_node = chi.server.create_server(
    "indyscc-head-node",                                 
    reservation_id=chi.lease.get_node_reservation(head_lease["id"]),
    image_name='CC-Ubuntu20.04',
    network_name="sharednet1",
    count=1)

In [ ]:
# Launch your compute node instances
compute_nodes = chi.server.create_server(
    "indyscc-compute-node",                                 
    reservation_id=chi.lease.get_node_reservation(compute_lease["id"]),
    image_name='CC-Ubuntu20.04',
    network_name="sharednet1",
    count=10)

In [ ]:
# wait for nodes to become active. This can take 10-20 minutes, as they need to reboot twice.
# you can run this again if it times out.
print(f"Waiting for instances to become active. Check out the current status at https://chi.rcac.purdue.edu/project/instances/")

head_node_start = chi.server.wait_for_active(head_node.id)

## Connecting to your server

By default, the server will only have a private IP assigned. In order to SSH to the server, you should assign a public floating IP. There are are limited amount of public IPs available across the entire Chameleon testbed, so try to keep the amount of nodes with a public IP to a minimum! A common practice is to set up one node as a "login node" with a public IP, and logging in to that node to manage all of your project's nodes.

Once a public IP is assigned, you can set up a remote connection to the node.

You can see your current floating IPs at https://chi.rcac.purdue.edu/project/floating_ips/

In [ ]:
from chi.ssh import Remote
# Assign a public floating IP
ip = chi.server.associate_floating_ip(head_node.id)

In [ ]:
# wait for ssh to become active
chi.server.wait_for_tcp(host=ip,port=22)

# Create a remote connection
remote = Remote(ip)

You can use `remote.run` to run commands on the server directly from your Notebook.
A similar command is `remote.sudo`, which runs a command with escalated privileges.

### Example command: get CPU usage

In [ ]:
# Run some commands on your remote server
cpu_usage = remote.run("grep 'cpu ' /proc/stat | awk '{usage=($2+$4)*100/($2+$4+$5)} END {print usage}'", hide=True)

print('Current CPU usage is {}%'.format(float(cpu_usage.stdout)))